# Example of using the IDR web API

OMERO.web uses a default session backend authentication scheme for authentication.
First create a HTTP session using the [`requests`](http://docs.python-requests.org/en/master/) library:

In [ ]:
# initial data
IDR_BASE_URL = "https://idr.openmicroscopy.org"


In [ ]:
import requests

INDEX_PAGE = "%s/webclient/?experimenter=-1" % IDR_BASE_URL

# create http session
with requests.Session() as session:
    request = requests.Request('GET', INDEX_PAGE)
    prepped = session.prepare_request(request)
    response = session.send(prepped)
    if response.status_code != 200:
        response.raise_for_status()

# Studies:

## Get Study map annotation:

In [ ]:
MAP_URL = "{base}/webclient/api/annotations/?type=map&{type}={screen_id}"

SCREEN_ID = 102

qs = {'base': IDR_BASE_URL, 'type': 'screen', 'screen_id': SCREEN_ID}
url = MAP_URL.format(**qs)
for a in session.get(url).json()['annotations']:
    namespace = a['ns']
    for v in a['values']:
        key = v[0]
        value = v[1]
        print (key, value)

## Get Plates in the given Screen:

In [ ]:
PLATES_URL = "{base}/webclient/api/plates/?id={screen_id}"

SCREEN_ID = 102

qs = {'base': IDR_BASE_URL, 'screen_id': SCREEN_ID}
url = PLATES_URL.format(**qs)
for p in session.get(url).json()['plates']:
    plate_id = p['id']
    print("Plate: {id}, Name: {name}, Children: {childCount}".format(**p))

## Get PlateGrid:

In [ ]:
WELLS_IMAGES_URL = "{base}/webgateway/plate/{plate_id}/{field}/"

qs = {'base': IDR_BASE_URL, 'plate_id': plate_id, 'field': 0}
url = WELLS_IMAGES_URL.format(**qs)
grid = session.get(url).json()
rowlabels = grid['rowlabels']
collabels = grid['collabels']
for row in grid['grid']:
    for cell in row:
        if cell is not None:
            print("Well: {wellId}, Image: {id}".format(**cell))


## Get Image Thumbnail:

In [ ]:
from IPython.display import display, HTML

WELLS_IMAGES_URL = "{base}/webgateway/plate/{plate_id}/{field}/"
PLATE_ID = 325

qs = {'base': IDR_BASE_URL, 'plate_id': PLATE_ID, 'field': 0}
url = WELLS_IMAGES_URL.format(**qs)
grid = session.get(url).json()

image_ids = [cell['id']  for row in grid['grid'] for cell in row if cell is not None]

THUMBNAILS_URL = "{base}/webgateway/get_thumbnails/".format(**{'base': IDR_BASE_URL, })

def thumbnails_chunk(image_list, batch):
    for b in range(0, len(image_list), batch):
        yield image_list[b:b + batch]
batch_size = 20

_thumbnails = {}
for batch in thumbnails_chunk(image_ids, batch_size):
    payload = {'id': batch}
    for iid, thumb in session.get(THUMBNAILS_URL, params=payload).json().items():
        _thumbnails[int(iid)] = thumb

# Display plate
_tbody = []
for row in grid['grid']:
    _tr = ["<tr>"]
    for cell in row:
        if cell is not None:
            _tr.append('<td><img src="%s"/></td>' % _thumbnails[cell['id']])
        else:
            _tr.append('<td></td>')
    _tr.append("</tr>")
    _tbody.append("&nbsp;".join(_tr))
display(HTML("<table>" + "".join(_tbody) + "</table>"))

## Get Image:

In [ ]:
from IPython.display import display, HTML

IMAGE_DETAILS_URL = "{base}/webclient/imgData/{image_id}/"

IMAGE_ID = 122770

qs = {'base': IDR_BASE_URL, 'image_id': IMAGE_ID}
url = IMAGE_DETAILS_URL.format(**qs)
r = session.get(url)
if r.status_code == 200:
    print (r.json())

RENDER_IMAGE = "{base}/webgateway/render_image/{image_id}/0/0/"
img_url = RENDER_IMAGE.format(**qs)

display(HTML("<img src='%s' />" % img_url))

## Get Image map annotation:

In [ ]:
MAP_URL = "{base}/webclient/api/annotations/?type=map&{type}={image_id}"

IMAGE_ID = 122770

qs = {'base': IDR_BASE_URL, 'type': 'image', 'image_id': IMAGE_ID}
url = MAP_URL.format(**qs)
for a in session.get(url).json()['annotations']:
    namespace = a['ns']
    for v in a['values']:
        key = v[0]
        value = v[1]
        print (key, value)

## Get bulk annotation:

In [ ]:
BULK_URL = "{base}/webgateway/table/Screen.plateLinks.child.wells/{well_id}/query/?query=Well-{well_id}"

WELL_ID = 45217
qs = {'base': IDR_BASE_URL, 'well_id': WELL_ID}
url = BULK_URL.format(**qs)
r = session.get(url)
print (r.json())


# or download entire bulk_annotation file:

FILEANNOTATION_URL = "{base}/webclient/api/annotations/?type=file&screen={screen_id}"
DOWNLOAD_URL = "{base}/webclient/annotation/{ann_id}"

SCREEN_ID = 206

qs = {'base': IDR_BASE_URL, 'screen_id': SCREEN_ID}
url = FILEANNOTATION_URL.format(**qs)
for a in session.get(url).json()['annotations']:
    namespace = a['ns']
    ann_id = a['id']
    qs2 = {'base': IDR_BASE_URL, 'ann_id':  a['id']}
    url2 = DOWNLOAD_URL.format(**qs2)
    print ("Download URL:", url2)

# Attributes (e.g. Gene, Phenotype...)

## Get Screens that are annotated with gene:

In [ ]:
SCREENS_PROJECTS_URL = "{base}/mapr/api/{key}/?value={value}"

qs = {'base': IDR_BASE_URL, 'key': 'gene', 'value': 'CDC20'}
url = SCREENS_PROJECTS_URL.format(**qs)
for s in session.get(url).json()['screens']:
    screen_id = s['id']
    print (s['id'], s['name'])

## Get Plates in Screen that are annotated with gene:

In [ ]:
PLATES_URL = "{base}/mapr/api/{key}/plates/?value={value}&id={screen_id}"

qs = {'base': IDR_BASE_URL, 'key': 'gene', 'value': 'CDC20', 'screen_id': screen_id}
url = PLATES_URL.format(**qs)
for p in session.get(url).json()['plates']:
    plate_id = p['id']
    print (p['id'], p['name'])

## Get Images in Plate that are annotated with gene:

In [ ]:
IMAGES_URL = "{base}/mapr/api/{key}/images/?value={value}&node={parent_type}&id={parent_id}"

IMAGE_URL = "{base}/webclient/?show=image-{image_id}"
IMAGE_VIEWER = "{base}/webclient/img_detail/{image_id}/"
THUMBNAIL_URL = "{base}/webclient/render_thumbnail/{image_id}/"
ATTRIBUTES_URL = "{base}/webclient/api/annotations/?type=map&image={image_id}"

qs = {'base': IDR_BASE_URL, 'key': 'gene', 'value': 'CDC20', 'parent_type': 'plate', 'parent_id': plate_id}
url = IMAGES_URL.format(**qs)
for i in session.get(url).json()['images']:
    image_id = i['id']
    print ('Image link:', IMAGE_URL.format(**{'base': IDR_BASE_URL, 'image_id': image_id}))
    print ('Image viewer link:', IMAGE_VIEWER.format(**{'base': IDR_BASE_URL, 'image_id': image_id}))
    print ('Thumbnail URL:', THUMBNAIL_URL.format(**{'base': IDR_BASE_URL, 'image_id': image_id}))
    url = ATTRIBUTES_URL.format(**{'base': IDR_BASE_URL, 'image_id': image_id})
    print ('Annotations:')
    for a in session.get(url).json()['annotations']:
        print ('\t%s' % a['values'])